# Classification and regression models

In [1]:
import rebound
from spock import FeatureClassifier
feature_model = FeatureClassifier()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.move_to_com()

print(feature_model.predict_stable(sim))

0.06591137


In [2]:
from spock import DeepRegressor
deep_model = DeepRegressor()

median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))

252078


In [3]:
from spock import class_MLP, CollisionMergerClassifier #for some reason, you need to import class_MLP here
class_model = CollisionMergerClassifier()

prob_12, prob_23, prob_13 = class_model.predict_collision_probs(sim)
print(prob_12, prob_23, prob_13)

0.27385116 0.49274087 0.23340799


In [4]:
from spock import reg_MLP, CollisionOrbitalOutcomeRegressor #for some reason, you need to import reg_MLP here
reg_model = CollisionOrbitalOutcomeRegressor()

a1, a2, e1, e2, inc1, inc2 = reg_model.predict_collision_outcome(sim, collision_inds=[2, 3]) #inclinations are measured wrt to L axis (coplanar is maybe a bad example...)
print(a1, a2, e1, e2, inc1, inc2)

0.3362391024283648 0.3697486909196322 0.05889093054898315 0.029298967242682977 0.007076251680723447 0.0033700382462669653


# Iterative planet formation predictions

In [5]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.add(m=1.e-5, P=2.2, e=0.03, pomega=0.5, l=5.0)
sim.add(m=1.e-5, P=2.5, e=0.03, pomega=5.0, l=1.5)
sim.move_to_com()

In [6]:
from spock import GiantImpactPhaseEmulator
PF_map = GiantImpactPhaseEmulator([sim])#, sim, sim]) # add 3 realizations

In [7]:
%%time
PF_map.integrate(tmax=1e9)

TypeError: 'float' object is not subscriptable

In [20]:
ps = PF_map.sims[0].particles
print('t = {0:.3e}'.format(PF_map.sims[0].t))
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')

t = 1.000e+09
m1: 1e-05
a1: 0.30884464930306393
e1: 0.08348373998523444
inc1: 0.016061431418397668 

m2: 3.0000000000000004e-05
a2: 0.40533759006174347
e2: 0.035807798937329785
inc2: 0.0042127913157672 

m3: 1e-05
a3: 0.615328376177772
e3: 0.06158931733559472
inc3: 0.008643987765806067 



In [21]:
ps = PF_map.sims[1].particles
print('t = {0:.3e}'.format(PF_map.sims[0].t))
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')

t = 1.000e+09
m1: 2e-05
a1: 0.2965574736680634
e1: 0.04867488015721477
inc1: 0.010104928301212136 

m2: 3.0000000000000004e-05
a2: 0.5124500536621529
e2: 0.03742784662423185
inc2: 0.005218944991968547 



In [22]:
ps = PF_map.sims[2].particles
print('t = {0:.3e}'.format(PF_map.sims[0].t))
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')

t = 1.000e+09
m1: 1e-05
a1: 0.3018202630023906
e1: 0.06818261855504561
inc1: 0.01397316842654979 

m2: 2e-05
a2: 0.38184515267778907
e2: 0.0410148406728638
inc2: 0.005027218298585059 

m3: 2e-05
a3: 0.5342055140821831
e3: 0.04039901823669516
inc3: 0.0020612863159267946 



In [23]:
PF_map = GiantImpactPhaseEmulator(sim)

In [24]:
PF_map.step(tmax=1e9)

ps = PF_map.sims[0].particles
print('t = {0:.3e}'.format(PF_map.sims[0].t))
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')

t = 6.053e+03
m1: 1e-05
a1: 0.29384466869379183
e1: 0.018660525240299743
inc1: 0.0 

m2: 1e-05
a2: 0.3311137797188888
e2: 0.015069371836333275
inc2: 0.0 

m3: 1e-05
a3: 0.4147639836309363
e3: 0.0702881128955157
inc3: 0.007388539213993718 

m4: 2e-05
a4: 0.5341822242074195
e4: 0.040385932527483404
inc4: 0.0032553387241999973 



In [25]:
PF_map.step(tmax=1e9)

ps = PF_map.sims[0].particles
print('t = {0:.3e}'.format(PF_map.sims[0].t))
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')

t = 2.034e+05
m1: 2e-05
a1: 0.35579753298812195
e1: 0.044863633280496217
inc1: 0.009177617757973827 

m2: 1e-05
a2: 0.35630608980693856
e2: 0.07596872977080286
inc2: 0.014199140575983272 

m3: 2e-05
a3: 0.5340823493798511
e3: 0.04032369967801774
inc3: 0.005707415011835588 



In [26]:
PF_map.step(tmax=1e9)

ps = PF_map.sims[0].particles
print('t = {0:.3e}'.format(PF_map.sims[0].t))
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')

t = 2.286e+05
m1: 1e-05
a1: 0.3925317667862876
e1: 0.13465081841183277
inc1: 0.028102185550548536 

m2: 4e-05
a2: 0.4460821741460833
e2: 0.040824800547423754
inc2: 0.006470574842037422 



In [27]:
PF_map.step(tmax=1e9)

ps = PF_map.sims[0].particles
print('t = {0:.3e}'.format(PF_map.sims[0].t))
for i in range(1, len(ps)):
    print('m' + str(i) + ':', ps[i].m)
    print('a' + str(i) + ':', ps[i].a)
    print('e' + str(i) + ':', ps[i].e)
    print('inc' + str(i) + ':', ps[i].inc, '\n')

t = 1.000e+09
m1: 1e-05
a1: 0.3925317667862876
e1: 0.13465081841183277
inc1: 0.028102185550548536 

m2: 4e-05
a2: 0.4460821741460833
e2: 0.040824800547423754
inc2: 0.006470574842037422 

